# 🚀 FasalVaidya Enhanced Training V2

**Key Features:**
- Binary Leaf Validator (rejects non-leaf images)
- PlantVillage Pre-training (smart download)
- Two-Phase Fine-Tuning (freeze → unfreeze)
- Confidence-Based Predictions (high/medium/low)
- RAM Optimized for Colab

**Training Time:** ~2 hours on T4 GPU

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Install and import packages
!pip install -q tensorflow>=2.15.0 kaggle opendatasets scikit-learn matplotlib seaborn

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import json
import shutil
import random
import gc
from pathlib import Path
from datetime import datetime
from sklearn.metrics import classification_report, confusion_matrix

print(f"TensorFlow: {tf.__version__}")
print(f"GPU: {tf.config.list_physical_devices('GPU')}")

np.random.seed(42)
tf.random.set_seed(42)
SESSION_START = datetime.now()
print(f"Session: {SESSION_START.strftime('%H:%M:%S')}")

In [ ]:
# =============================================================
# CONFIGURATION - RAM OPTIMIZED
# =============================================================
IMG_SIZE = 224
BATCH_SIZE = 16  # Reduced for RAM

# Epochs
LEAF_VALIDATOR_EPOCHS = 5
PLANTVILLAGE_EPOCHS = 6
FROZEN_EPOCHS = 6
FINETUNE_EPOCHS = 4

# Learning rates
LR_FROZEN = 1e-3
LR_FINETUNE = 1e-5

# Confidence thresholds
CONF_HIGH = 0.85
CONF_LOW = 0.50

# Paths
OUTPUT_DIR = '/content/fasalvaidya_enhanced'
DRIVE_CHECKPOINT = '/content/drive/MyDrive/FasalVaidya_Enhanced_Checkpoints'
PLANTVILLAGE_PATH = '/content/plantvillage'
NUTRIENT_ROOT = '/content/local_nutrients'
UNIFIED_PATH = '/content/unified_dataset'
LEAF_DATA_PATH = '/content/leaf_validation_data'

os.makedirs(OUTPUT_DIR, exist_ok=True)
os.makedirs(DRIVE_CHECKPOINT, exist_ok=True)

# GPU config - Memory optimized
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)

tf.config.optimizer.set_jit(False)  # Disable JIT to save RAM
tf.keras.mixed_precision.set_global_policy('float32')

# Crops
CROP_DATASETS = {
    'rice': 'Rice Nutrients',
    'wheat': 'Wheat Nitrogen',
    'maize': 'Maize Nutrients',
    'banana': 'Banana leaves Nutrient',
    'coffee': 'Coffee Nutrients',
    'ashgourd': 'Ashgourd Nutrients',
    'eggplant': 'EggPlant Nutrients',
    'snakegourd': 'Snakegourd Nutrients',
    'bittergourd': 'Bittergourd Nutrients',
}

print(f"✅ Config: {len(CROP_DATASETS)} crops, batch={BATCH_SIZE}")

## 🔑 Kaggle & PlantVillage (Smart Download)

In [ ]:
# Smart detection
KAGGLE_CONFIG = os.path.expanduser('~/.kaggle/kaggle.json')
USE_PLANTVILLAGE = True

def check_kaggle():
    if os.path.exists(KAGGLE_CONFIG):
        try:
            with open(KAGGLE_CONFIG) as f:
                if 'username' in f.read():
                    return True
        except: pass
    return False

def find_plantvillage():
    paths = ['/content/plantvillage/plantdisease/PlantVillage',
             '/content/plantvillage/PlantVillage',
             '/content/drive/MyDrive/PlantVillage']
    for p in paths:
        if os.path.exists(p) and len(os.listdir(p)) >= 15:
            return p
    return None

kaggle_ok = check_kaggle()
pv_path = find_plantvillage()
print(f'Kaggle: {"✅" if kaggle_ok else "❌"} | PlantVillage: {"✅ " + pv_path if pv_path else "❌"}')

In [ ]:
# Kaggle setup if needed
if USE_PLANTVILLAGE and not pv_path and not kaggle_ok:
    print('📤 Upload kaggle.json (https://kaggle.com/settings → API → Create Token)')
    from google.colab import files
    uploaded = files.upload()
    if 'kaggle.json' in uploaded:
        !mkdir -p ~/.kaggle && mv kaggle.json ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json
        print('✅ Kaggle configured')
        kaggle_ok = True
    else:
        USE_PLANTVILLAGE = False
elif pv_path:
    print('✅ PlantVillage exists, skipping setup')
else:
    print('✅ Kaggle ready')

In [ ]:
# Download PlantVillage if needed
if USE_PLANTVILLAGE:
    pv_path = find_plantvillage()
    if pv_path:
        PLANTVILLAGE_PATH = pv_path
        print(f'✅ Using existing: {PLANTVILLAGE_PATH}')
    else:
        print('📥 Downloading PlantVillage (~3-5 min)...')
        import opendatasets as od
        os.makedirs('/content/plantvillage', exist_ok=True)
        od.download('https://www.kaggle.com/datasets/emmarex/plantdisease', data_dir='/content/plantvillage')
        pv_path = find_plantvillage()
        if pv_path:
            PLANTVILLAGE_PATH = pv_path
            print(f'✅ Downloaded: {len(os.listdir(PLANTVILLAGE_PATH))} classes')
        else:
            print('⚠️ Download failed, skipping')
            USE_PLANTVILLAGE = False
else:
    print('ℹ️ PlantVillage skipped')

## 📁 Nutrient Dataset

In [ ]:
# Find nutrient dataset
search = ['/content/drive/MyDrive/Leaf Nutrient Data Sets']
shortcut_dir = '/content/drive/MyDrive/.shortcut-targets-by-id'
if os.path.exists(shortcut_dir):
    for fid in os.listdir(shortcut_dir):
        t = os.path.join(shortcut_dir, fid, 'Leaf Nutrient Data Sets')
        if os.path.exists(t): search.append(t)

DRIVE_NUTRIENT = None
for p in search:
    if os.path.exists(p) and len(os.listdir(p)) >= 5:
        DRIVE_NUTRIENT = p
        break

if not DRIVE_NUTRIENT:
    raise FileNotFoundError('Dataset not found! Add shortcut to My Drive.')
print(f'✅ Found: {DRIVE_NUTRIENT}')

In [ ]:
# Copy to local SSD
os.makedirs(NUTRIENT_ROOT, exist_ok=True)
print('Copying to SSD...')
for crop, folder in CROP_DATASETS.items():
    src = os.path.join(DRIVE_NUTRIENT, folder)
    dst = os.path.join(NUTRIENT_ROOT, folder)
    if os.path.exists(dst) and sum(1 for _ in Path(dst).rglob('*.jpg')) > 50:
        print(f'  {crop}: exists')
        continue
    if os.path.exists(src):
        print(f'  {crop}: copying...', end=' ')
        if os.path.exists(dst): shutil.rmtree(dst)
        shutil.copytree(src, dst)
        print('✅')
print(f'✅ Data on SSD')

## 🌿 Part 1: Leaf Validator

In [ ]:
# Create leaf validation data
os.makedirs(f'{LEAF_DATA_PATH}/leaf', exist_ok=True)
os.makedirs(f'{LEAF_DATA_PATH}/non_leaf', exist_ok=True)

# Collect leaf samples
print('Collecting leaves...')
leaf_count = 0
for crop, folder in CROP_DATASETS.items():
    crop_path = os.path.join(NUTRIENT_ROOT, folder)
    if not os.path.exists(crop_path): continue
    for root, dirs, files in os.walk(crop_path):
        imgs = [f for f in files if f.lower().endswith(('.jpg', '.png'))][:20]
        for img in imgs:
            shutil.copy2(os.path.join(root, img), f'{LEAF_DATA_PATH}/leaf/{crop}_{leaf_count}_{img}')
            leaf_count += 1
        if leaf_count > 150: break
print(f'✅ {leaf_count} leaf images')

# Generate non-leaf
from PIL import Image
for i in range(leaf_count):
    if i % 4 == 0:
        img = Image.new('RGB', (224,224), (random.randint(0,255), random.randint(0,255), random.randint(0,255)))
    elif i % 4 == 1:
        img = Image.fromarray(np.random.randint(0, 256, (224,224,3), dtype=np.uint8))
    elif i % 4 == 2:
        arr = np.zeros((224,224,3), dtype=np.uint8)
        for y in range(224): arr[y,:,:] = [y, 128, 255-y]
        img = Image.fromarray(arr)
    else:
        arr = np.clip(np.random.randint(-20, 20, (224,224,3)) + [139,90,43], 0, 255).astype(np.uint8)
        img = Image.fromarray(arr)
    img.save(f'{LEAF_DATA_PATH}/non_leaf/syn_{i:04d}.jpg')
print(f'✅ {leaf_count} non-leaf images')

In [ ]:
# Create datasets - save class_names BEFORE transforming
train_leaf_raw = tf.keras.utils.image_dataset_from_directory(
    LEAF_DATA_PATH, validation_split=0.2, subset='training', seed=42,
    image_size=(IMG_SIZE, IMG_SIZE), batch_size=BATCH_SIZE, label_mode='binary')
val_leaf_raw = tf.keras.utils.image_dataset_from_directory(
    LEAF_DATA_PATH, validation_split=0.2, subset='validation', seed=42,
    image_size=(IMG_SIZE, IMG_SIZE), batch_size=BATCH_SIZE, label_mode='binary')

leaf_class_names = train_leaf_raw.class_names  # Save before transform!

def norm(img, lbl):
    return tf.keras.applications.mobilenet_v2.preprocess_input(tf.cast(img, tf.float32)), lbl

train_leaf = train_leaf_raw.map(norm).prefetch(tf.data.AUTOTUNE)
val_leaf = val_leaf_raw.map(norm).prefetch(tf.data.AUTOTUNE)
print(f'Leaf classes: {leaf_class_names}')

In [ ]:
# Build and train leaf validator
base_v = tf.keras.applications.MobileNetV2(input_shape=(IMG_SIZE,IMG_SIZE,3), include_top=False, weights='imagenet')
base_v.trainable = False

leaf_validator = tf.keras.Sequential([
    base_v,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid', dtype='float32')
], name='leaf_validator')

leaf_validator.compile(optimizer=tf.keras.optimizers.Adam(LR_FROZEN), loss='binary_crossentropy', metrics=['accuracy'])

print(f'🌿 Training Leaf Validator ({LEAF_VALIDATOR_EPOCHS} epochs)')
h_leaf = leaf_validator.fit(train_leaf, validation_data=val_leaf, epochs=LEAF_VALIDATOR_EPOCHS,
    callbacks=[tf.keras.callbacks.EarlyStopping(patience=2, restore_best_weights=True),
               tf.keras.callbacks.ModelCheckpoint(f'{OUTPUT_DIR}/leaf_validator.keras', save_best_only=True)])
leaf_acc = max(h_leaf.history['val_accuracy'])
print(f'✅ Leaf Validator: {leaf_acc:.1%}')
gc.collect()

## 🌱 Part 2a: PlantVillage Pre-training (Optional)

In [ ]:
PRETRAINED_BASE = None
if USE_PLANTVILLAGE and os.path.exists(PLANTVILLAGE_PATH):
    print(f'🌱 PlantVillage ({PLANTVILLAGE_EPOCHS} epochs)')
    train_pv_raw = tf.keras.utils.image_dataset_from_directory(
        PLANTVILLAGE_PATH, validation_split=0.2, subset='training', seed=42,
        image_size=(IMG_SIZE, IMG_SIZE), batch_size=BATCH_SIZE, label_mode='categorical')
    val_pv_raw = tf.keras.utils.image_dataset_from_directory(
        PLANTVILLAGE_PATH, validation_split=0.2, subset='validation', seed=42,
        image_size=(IMG_SIZE, IMG_SIZE), batch_size=BATCH_SIZE, label_mode='categorical')
    
    pv_classes = train_pv_raw.class_names
    train_pv = train_pv_raw.map(norm).prefetch(tf.data.AUTOTUNE)
    val_pv = val_pv_raw.map(norm).prefetch(tf.data.AUTOTUNE)
    
    base_pv = tf.keras.applications.MobileNetV2(input_shape=(IMG_SIZE,IMG_SIZE,3), include_top=False, weights='imagenet')
    base_pv.trainable = False
    model_pv = tf.keras.Sequential([base_pv, tf.keras.layers.GlobalAveragePooling2D(),
        tf.keras.layers.Dropout(0.3), tf.keras.layers.Dense(256, activation='relu'),
        tf.keras.layers.Dense(len(pv_classes), activation='softmax', dtype='float32')])
    model_pv.compile(optimizer=tf.keras.optimizers.Adam(LR_FROZEN), loss='categorical_crossentropy', metrics=['accuracy'])
    
    h_pv = model_pv.fit(train_pv, validation_data=val_pv, epochs=PLANTVILLAGE_EPOCHS,
        callbacks=[tf.keras.callbacks.EarlyStopping(patience=2, restore_best_weights=True)])
    print(f'✅ PlantVillage: {max(h_pv.history["val_accuracy"]):.1%}')
    PRETRAINED_BASE = base_pv
    del train_pv, val_pv, train_pv_raw, val_pv_raw, model_pv
    gc.collect()
else:
    print('ℹ️ Skipping PlantVillage')

## 🌾 Part 2b: Disease Classifier

In [ ]:
# Build unified dataset - handles all folder structures
if os.path.exists(UNIFIED_PATH): shutil.rmtree(UNIFIED_PATH)
os.makedirs(UNIFIED_PATH)

CLASS_RENAME = {
    'rice': {'Nitrogen(N)': 'rice_nitrogen', 'Phosphorus(P)': 'rice_phosphorus', 'Potassium(K)': 'rice_potassium'},
    'wheat': {'control': 'wheat_control', 'deficiency': 'wheat_deficiency'},
    'maize': {'ALL Present': 'maize_all_present', 'ALLAB': 'maize_allab', 'KAB': 'maize_kab', 'NAB': 'maize_nab', 'PAB': 'maize_pab', 'ZNAB': 'maize_znab'},
    'banana': {'healthy': 'banana_healthy', 'magnesium': 'banana_magnesium', 'potassium': 'banana_potassium'},
    'coffee': {'healthy': 'coffee_healthy', 'nitrogen-N': 'coffee_nitrogen', 'phosphorus-P': 'coffee_phosphorus', 'potasium-K': 'coffee_potassium'},
}

def get_cls(crop, orig):
    if crop in CLASS_RENAME and orig in CLASS_RENAME[crop]: return CLASS_RENAME[crop][orig]
    return f"{crop}_{orig.lower().replace(' ', '_').replace('-', '_')}"

def copy_imgs(src, dst, prefix):
    cnt = 0
    if not os.path.exists(src): return 0
    for f in os.listdir(src):
        if f.lower().endswith(('.jpg', '.jpeg', '.png')):
            shutil.copy2(os.path.join(src, f), os.path.join(dst, f'{prefix}_{f}'))
            cnt += 1
    return cnt

print('Building unified dataset...')
class_counts = {}
for crop, folder in CROP_DATASETS.items():
    crop_path = os.path.join(NUTRIENT_ROOT, folder)
    if not os.path.exists(crop_path): continue
    crop_imgs = 0
    for item in os.listdir(crop_path):
        item_path = os.path.join(crop_path, item)
        if not os.path.isdir(item_path): continue
        
        if item.lower() in ['train', 'test', 'val', 'validation']:
            for cls in os.listdir(item_path):
                cls_path = os.path.join(item_path, cls)
                if os.path.isdir(cls_path):
                    uc = get_cls(crop, cls)
                    dst = os.path.join(UNIFIED_PATH, uc)
                    os.makedirs(dst, exist_ok=True)
                    n = copy_imgs(cls_path, dst, f'{crop}_{item}')
                    class_counts[uc] = class_counts.get(uc, 0) + n
                    crop_imgs += n
        else:
            uc = get_cls(crop, item)
            dst = os.path.join(UNIFIED_PATH, uc)
            os.makedirs(dst, exist_ok=True)
            subs = os.listdir(item_path)
            if any(s.lower() in ['train', 'test', 'val', 'validation'] for s in subs):
                for sp in subs:
                    sp_path = os.path.join(item_path, sp)
                    if os.path.isdir(sp_path):
                        n = copy_imgs(sp_path, dst, f'{crop}_{item}_{sp}')
                        class_counts[uc] = class_counts.get(uc, 0) + n
                        crop_imgs += n
            else:
                n = copy_imgs(item_path, dst, crop)
                class_counts[uc] = class_counts.get(uc, 0) + n
                crop_imgs += n
    print(f'  {crop}: {crop_imgs}')

class_names = sorted(class_counts.keys())
num_classes = len(class_names)
total_images = sum(class_counts.values())
print(f'\n✅ {num_classes} classes, {total_images:,} images')

In [ ]:
# Create datasets
class_weight = {i: total_images / (num_classes * class_counts.get(c, 1)) for i, c in enumerate(class_names)}

train_raw = tf.keras.utils.image_dataset_from_directory(
    UNIFIED_PATH, validation_split=0.2, subset='training', seed=42,
    image_size=(IMG_SIZE, IMG_SIZE), batch_size=BATCH_SIZE, label_mode='categorical')
val_raw = tf.keras.utils.image_dataset_from_directory(
    UNIFIED_PATH, validation_split=0.2, subset='validation', seed=42,
    image_size=(IMG_SIZE, IMG_SIZE), batch_size=BATCH_SIZE, label_mode='categorical')

class_names = train_raw.class_names
num_classes = len(class_names)

aug = tf.keras.Sequential([tf.keras.layers.RandomFlip('horizontal'),
    tf.keras.layers.RandomRotation(0.05), tf.keras.layers.RandomZoom(0.1),
    tf.keras.layers.RandomBrightness(0.1), tf.keras.layers.RandomContrast(0.1)])

@tf.function
def aug_norm(img, lbl): return tf.keras.applications.mobilenet_v2.preprocess_input(aug(img, training=True)), lbl
@tf.function
def norm_only(img, lbl): return tf.keras.applications.mobilenet_v2.preprocess_input(tf.cast(img, tf.float32)), lbl

train_ds = train_raw.map(aug_norm, num_parallel_calls=tf.data.AUTOTUNE).prefetch(tf.data.AUTOTUNE)
val_ds = val_raw.map(norm_only, num_parallel_calls=tf.data.AUTOTUNE).prefetch(tf.data.AUTOTUNE)
print(f'✅ {num_classes} classes ready')

In [ ]:
# Build disease model
if PRETRAINED_BASE is not None:
    print('Using PlantVillage base')
    base_model = PRETRAINED_BASE
else:
    print('Using ImageNet base')
    base_model = tf.keras.applications.MobileNetV2(input_shape=(IMG_SIZE,IMG_SIZE,3), include_top=False, weights='imagenet')

base_model.trainable = False

disease_model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(256, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(1e-4)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(num_classes, activation='softmax', dtype='float32')
], name='disease_classifier')

disease_model.compile(optimizer=tf.keras.optimizers.Adam(LR_FROZEN), loss='categorical_crossentropy',
    metrics=['accuracy', tf.keras.metrics.TopKCategoricalAccuracy(k=3, name='top3')])
print(f'✅ Model ready: {num_classes} classes')

In [ ]:
# Phase 1: Frozen
print(f'\n🔒 Phase 1: Frozen ({FROZEN_EPOCHS} epochs)')
h1 = disease_model.fit(train_ds, validation_data=val_ds, epochs=FROZEN_EPOCHS, class_weight=class_weight,
    callbacks=[tf.keras.callbacks.EarlyStopping(patience=3, restore_best_weights=True),
               tf.keras.callbacks.ReduceLROnPlateau(factor=0.5, patience=2)])
p1_acc = max(h1.history['val_accuracy'])
print(f'✅ Phase 1: {p1_acc:.1%}')
gc.collect()

In [ ]:
# Phase 2: Fine-tune
print(f'\n🔓 Phase 2: Fine-tune ({FINETUNE_EPOCHS} epochs)')
base_model.trainable = True
for layer in base_model.layers[:-30]: layer.trainable = False

disease_model.compile(optimizer=tf.keras.optimizers.Adam(LR_FINETUNE), loss='categorical_crossentropy',
    metrics=['accuracy', tf.keras.metrics.TopKCategoricalAccuracy(k=3, name='top3')])

h2 = disease_model.fit(train_ds, validation_data=val_ds, epochs=FINETUNE_EPOCHS, class_weight=class_weight,
    callbacks=[tf.keras.callbacks.EarlyStopping(patience=2, restore_best_weights=True),
               tf.keras.callbacks.ModelCheckpoint(f'{OUTPUT_DIR}/disease_final.keras', save_best_only=True),
               tf.keras.callbacks.ModelCheckpoint(f'{DRIVE_CHECKPOINT}/disease_final.keras', save_best_only=True)])
p2_acc = max(h2.history['val_accuracy'])
print(f'✅ Phase 2: {p2_acc:.1%}')
gc.collect()

## 📊 Evaluation

In [ ]:
# Evaluate
print('Evaluating...')
y_true, y_pred, y_probs = [], [], []
for imgs, lbls in val_ds:
    preds = disease_model.predict(imgs, verbose=0)
    y_true.extend(np.argmax(lbls.numpy(), axis=1))
    y_pred.extend(np.argmax(preds, axis=1))
    y_probs.extend(np.max(preds, axis=1))

y_true, y_pred, y_probs = np.array(y_true), np.array(y_pred), np.array(y_probs)
acc = (y_true == y_pred).mean()
print(f'\n📊 Accuracy: {acc:.1%}')

high = (y_probs >= CONF_HIGH).sum()
med = ((y_probs >= CONF_LOW) & (y_probs < CONF_HIGH)).sum()
low = (y_probs < CONF_LOW).sum()
print(f'Confidence: High {high/len(y_probs):.0%} | Med {med/len(y_probs):.0%} | Low {low/len(y_probs):.0%}')

## 💾 Export

In [ ]:
# Export leaf validator
print('Exporting leaf validator...')
conv = tf.lite.TFLiteConverter.from_keras_model(leaf_validator)
conv.optimizations = [tf.lite.Optimize.DEFAULT]
tfl = conv.convert()
with open(f'{OUTPUT_DIR}/leaf_validator.tflite', 'wb') as f: f.write(tfl)
with open(f'{DRIVE_CHECKPOINT}/leaf_validator.tflite', 'wb') as f: f.write(tfl)
print(f'✅ {len(tfl)/1024/1024:.1f} MB')

In [ ]:
# Export disease model
print('Exporting disease model...')
tf.keras.backend.clear_session()
best = tf.keras.models.load_model(f'{OUTPUT_DIR}/disease_final.keras')
_ = best(tf.zeros((1,224,224,3), dtype=tf.float32), training=False)

@tf.function(input_signature=[tf.TensorSpec([1,224,224,3], tf.float32)])
def serve(x): return best(x, training=False)

conv = tf.lite.TFLiteConverter.from_concrete_functions([serve.get_concrete_function()])
conv.optimizations = [tf.lite.Optimize.DEFAULT]
try: tfl = conv.convert()
except:
    conv.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS, tf.lite.OpsSet.SELECT_TF_OPS]
    tfl = conv.convert()

with open(f'{OUTPUT_DIR}/fasalvaidya_enhanced.tflite', 'wb') as f: f.write(tfl)
with open(f'{DRIVE_CHECKPOINT}/fasalvaidya_enhanced.tflite', 'wb') as f: f.write(tfl)
print(f'✅ {len(tfl)/1024/1024:.1f} MB')

In [ ]:
# Save metadata
meta = {'version': '2.0', 'date': datetime.now().isoformat(), 'classes': num_classes,
        'class_names': class_names, 'thresholds': {'high': CONF_HIGH, 'low': CONF_LOW},
        'accuracy': {'overall': float(acc), 'p1': float(p1_acc), 'p2': float(p2_acc)},
        'crops': list(CROP_DATASETS.keys())}
for d in [OUTPUT_DIR, DRIVE_CHECKPOINT]:
    with open(f'{d}/metadata.json', 'w') as f: json.dump(meta, f, indent=2)
    with open(f'{d}/labels.txt', 'w') as f: f.write('\n'.join(class_names))

print(f'\n✅ DONE! Time: {datetime.now() - SESSION_START}')
print(f'Files: {OUTPUT_DIR}')